In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

# Review the training data 
train_df = pd.read_csv('/kaggle/input/histopathologic-cancer-detection/train_labels.csv')

train_df['id'] = train_df['id'] + '.tif'
train_df['label'] = train_df['label'].astype(str)

print('Train Set Size:', train_df.shape)
print(train_df.head(5))

# Check label distribution 
sns.countplot(data=train_df, x='label', palette='viridis')
plt.title('Distribution of Label Values')
plt.xlabel('Label')
plt.ylabel('Count')
plt.show()

# Look at the images
image_dir = '/kaggle/input/histopathologic-cancer-detection/train/' 

# Load and display a few sample images
sample_images = train_df['id'].head(5).tolist()  # Get the first 5 image IDs from the dataset

# Create a plot to display the images
plt.figure(figsize=(15, 5))  # Adjust figure size
for idx, image_name in enumerate(sample_images):
    image_path = os.path.join(image_dir, image_name)
    if os.path.exists(image_path):
        img = Image.open(image_path)  # Load the image
        plt.subplot(1, 5, idx + 1)
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"Label: {train_df['label'].iloc[idx]}")
    else:
        print(f"Image not found: {image_path}")
plt.tight_layout()
plt.show()

STEP 1: Brief Description 

The goal of the project is to correctly predict if an image contines at least one pixel of tumor tissue. These labels can be used to detect cancer in patient images. The data contains test and train folders of .tif files which are the images that will be assessed. There is also a file with the training labels that has 220,025 labeled images. The distribution of images with label 0 and label 1 is not even with there being more 0 labeled images (see histogram above). However, this shows us that there are no unexpected or missing labels in the dataset. The only cleansing that might be necessary is for the ID column. It looks like all the letters are lowercase and they all end with '.tif.' That suffix may not be necessary but it should be fine to continue with it for now. 

STEP 2: Plan for analysis 
1. Preprocessing step - load the images into arrays and perform data augmentation (maybe flipping or scaling)
2. Normalize the pixel values
3. Train-Test split the training data
4. Use Convolutional Neural Network for the image classification
5. Research some options for thise like ResNet, VGG or MoblieNetV2

STEP 3: Model Architecture 
Comparing ResNet, VGG and MobileNet, I am choosing to start with MoblieNet as it requires fewer parameters while still achiving high accuracy. Out of the other options I researched, MoblieNetV2 seems to offer the highest standard and is speedy. It also is on the lower end of resources usage which will be helpful since I'm just using what is available in the Kaggle notebook. 

VGG has an intutitive design and general simplicity, but the high parameters and resource usage make me worry that it won't be a great fit for this project. ResNet is a solid second choice I will try if the MoblieNet option doesn't work as well as expected but I still feel like MobileNet should be able to get high accuracy for lower parameters/resource power compared to ResNet. 

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# Paths
train_csv = '/kaggle/input/histopathologic-cancer-detection/train_labels.csv'
train_dir = '/kaggle/input/histopathologic-cancer-detection/train/'
test_dir = '/kaggle/input/histopathologic-cancer-detection/test/'

# Load the data
train_df = pd.read_csv(train_csv)
train_df['id'] = train_df['id'] + '.tif'

# Split into training and validation
train, val = train_test_split(train_df, test_size=0.2, stratify=train_df['label'], random_state=42)

# Convert label column to string
train['label'] = train['label'].astype(str)
val['label'] = val['label'].astype(str)

# Data Augmentation and Generators
datagen_train = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

datagen_val = ImageDataGenerator(rescale=1./255)


train_gen = datagen_train.flow_from_dataframe(
    train,
    directory=train_dir,
    x_col='id',
    y_col='label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

val_gen = datagen_val.flow_from_dataframe(
    val,
    directory=train_dir,
    x_col='id',
    y_col='label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

In [ ]:
# Model Definition
base_model = MobileNetV2(weights=None, include_top=False, input_shape=(224, 224, 3))
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Model Training
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    callbacks=[early_stopping]
)

# Evaluate Model
val_loss, val_accuracy = model.evaluate(val_gen)
print(f"Validation Accuracy: {val_accuracy:.2f}")

# Load and Predict on Test Data
test_images = os.listdir(test_dir)
test_images_full = [os.path.join(test_dir, img) for img in test_images]

# Prepare test data for prediction
test_data = datagen_val.flow_from_dataframe(
    pd.DataFrame({'id': test_images}),
    directory=test_dir,
    x_col='id',
    y_col=None,
    target_size=(224, 224),
    batch_size=32,
    class_mode=None,
    shuffle=False
)

predictions = model.predict(test_data)
predicted_labels = (predictions > 0.5).astype(int)

# Prepare for Submission
submission = pd.DataFrame({
    'id': test_images,
    'label': predicted_labels.flatten()
})

submission.to_csv('/kaggle/working/submission.csv', index=False)
